## Titanic_2
**[링크](https://kaggle-kr.tistory.com/18?category=868316)**

In [2]:
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# plotly -> modern chart 만들기
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

df_train = pd.read_csv("../input/titanic_train.csv")
df_test = pd.read_csv("../input/titanic_test.csv")
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 # 자신을 더하기
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

df_test[df_test['Fare'].isnull()]['Fare'] = df_test['Fare'].mean()

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

## 3. Feature engineering
- null data 채우기 -> feature의 통계량 또는 다른 아이디어. 어떻게 채우느냐에 따라 모델 성능이 좌지우지됨
- 실제 모델의 학습에 쓰려고 하는 것이므로, train과 test 모두에 똑같이 적용해야 함!!

### 3.1 null 채우기
#### 3.1.1 Age null 채우기
- Age는 null이 20% 정도 됨. title(Mr., Mrs. 등) + statistics 통해 채우기

In [3]:
# 알파벳 + .까지 추출하기
df_train['Initial'] = df_train['Name'].str.extract('([A-Za-z]+)\.')
df_test['Initial'] = df_test['Name'].str.extract('([A-Za-z]+)\.')

In [26]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(
cmap='summer_r')

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [37]:
# 남/여가 구분되는게 많음!! 위 테이블 참고하여 남/여가 쓰는 initial 구분해 보기
# 이렇게 무식하게 말고, crosstab에서 한쪽 카운트가 0이면 구분할 수 없나?
sex_init = pd.crosstab(df_train['Initial'], df_train['Sex']).T
sex_init

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [40]:
for col in sex_init.columns:
    if sex_init.loc['female', col] == 0:
        df_train[df_train['Initial'] == col]['Initial']

Capt test
Col test
Don test
Jonkheer test
Major test
Master test
Mr test
Rev test
Sir test
